In [1]:
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
#fucking neat! 

In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("Len of shakespeare dataset in chars:",len(text))

Len of shakespeare dataset in chars: 1115394


In [5]:
#First thousand chars:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
#A mapping from chars to ints (Character level tokenizer)
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #encoder! feed it a string, shits out a vector of ints
decode = lambda d: ''.join([itos[c] for c in d]) #decoder! feed it a vector of ints, shits out a string

In [8]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
#Train, validation split. 
n = int(0.9*len(data)) #first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [10]:
#block size/context length
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When the input is {context}, the target is :{target}")

When the input is tensor([18]), the target is :47
When the input is tensor([18, 47]), the target is :56
When the input is tensor([18, 47, 56]), the target is :57
When the input is tensor([18, 47, 56, 57]), the target is :58
When the input is tensor([18, 47, 56, 57, 58]), the target is :1
When the input is tensor([18, 47, 56, 57, 58,  1]), the target is :15
When the input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is :47
When the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is :58


In [12]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    #generate a small batch of data inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print("-----------------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"When the input is {context}, the target is :{target}")
        

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----------------
When the input is tensor([24]), the target is :43
When the input is tensor([24, 43]), the target is :58
When the input is tensor([24, 43, 58]), the target is :5
When the input is tensor([24, 43, 58,  5]), the target is :57
When the input is tensor([24, 43, 58,  5, 57]), the target is :1
When the input is tensor([24, 43, 58,  5, 57,  1]), the target is :46
When the input is tensor([24, 43, 58,  5, 57,  1, 46]), the target is :43
When the input is tensor([24, 43, 58,  5, 57,  1, 46, 43]), the target is :39
When the input is tensor([44]), the target is :53
When the input is tensor([44,

In [13]:
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [14]:
vocab_size

65

In [15]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self,idx,targets = None):
        #idx and targets are both (B,T) tensors of integers:
        logits = self.token_embedding_table(idx) #(B,T,C) Batch_size = 4, Time = block_size = 8, channel = vocab_size = 65
        if targets == None:
            loss = None
        else:
            #loss2 = F.cross_entropy(logits,targets) #this by itself doesn't work. cross_entropy expects B C T, not B T C
            B, T , C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        #idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self(idx)
            #focus only on the last time step
            logits = logits[:,-1,:] #Becomes (B,C)
            #apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) #(B,C)
            #sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            #append sampled index to the running sequence
            idx = torch.cat((idx,idx_next), dim = 1) #(B, T+1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [16]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [17]:
#create a pytorch optimizer 
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [18]:
batch_size = 32
for steps in range(10000):
    #sample a batch of data
    xb, yb = get_batch("train")

    #evaluate the loss
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5727508068084717


In [19]:
#bigram shakespeare: 
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers je are!-e!
QLYotouciullle'z,
Thitertho s?
NDan'spererfo cist ripl chys er orlese;
Yo jehof h hecere ek? wferommot mowo soaf yoit, ince his, t, f at. fal whetrimy bupof tor atha Bu!
JOutho f cimimave.
NEDUSt cir selle p wie wede
Ro n apenor f'Y tover witys an sh d w t e w!
CEOntiretoaveE IINpe, theck. cung.
ORIsthies hacin benqurd bll, d a r w wistatsowor ath
Fivet bloll ang a-I theeancu,
LINCI'T:
Sarry t I Ane sze t
LCKI thit,
n.
Faure ds ppplirn!
meftou ow pring, avewist th;
TENTEMETCI gienco, An he waro whiougou he s i

In [20]:
#i trained a bigram language model on 40 thousand lines of shakespeare and this is what it looks like 

#WE CAN MAKE THIS BETTER. TRANSFORMER INCOMING!!!!!

### Neat math trick in self-attention

In [21]:
#Consider the following toy-example: 
torch.manual_seed(1337)

B,T,C = 4,8,2 #batch, time, channels

x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [22]:
#we want x[b,t] = mean_{i<=t} x[b,i] 
#Version 1: Bag of words! vertical running average. Inefficient. We can do this with matmuls of normalized triangular matrices. See below: 
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [23]:
#Version 2! 
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(1,keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [24]:
xbow2 = wei @ x # (T,T) @ (B,T,C) ----> (B,T,T) @ (B,T,C) [A batched matmul]

In [25]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei #before softmax

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [26]:
#Version 3! Use Softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x

In [27]:
wei #after softmax

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [46]:
#Version 4: self-attention! 
torch.manual_seed(1337)
B,T,C = 4,8, 32

x = torch.randn(B,T,C)

#let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B,T, head_size = 16)
q = query(x) # (B,T, head_size = 16)

wei = q @ k.transpose(-2,-1) #Doesn't make sense to transpose a 3x3 tensor, but we got batches of TxC's so we only transpose over the last 2 dims. resultant dim = (B,T,16) @ (B,16,T) = (B,T,T)
#Now all the weights are data dependent! 

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T)) #originally done; this results in uniform connections between each token. we don't neccesarily want this, we want each token to be connected to other tokens based 
                         # on some kind of heuristic ("which one is best?") and this is what self-attention solves. It's done by a query and a key vector for each token, the query contains 
                         # "what we're looking for" for the respective token & the key contains "what we have". We dot product the query for our token with the keys to all
                         # the other tokens to calculate "affinities", so to speak. mathematically if our key & query vectors are aligned in our language embedding space the dot prod will
                         # be higher & we know which tokens to prioritize. We implement a single "head" of self-attention
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim =1)
v = value(x)
out = wei @ v

#out = wei @ x 

out.shape
#Note on attention: There is no notion of apace, attention simply acts over a set of vectors. That's why we need to positionally encode tokens. 
#Each example across batch dimensions is processed completely independently and never "talk" to one another
#In an "encoder" attention block (for sentiment analysis for example), delete the single line that does the masking with tril. This is only done in this case because we want to use
#previous tokens to predict the next token, so it doesn't make sense for us to have weighted affinities for something we're trying to predict. 
#What we're doing here is called a decoder block. 

#Self attention just means that the keys queries & values all come from the same source, x. but in principle attention dont care. It's much more general. For encoder-decoder 
#transformers, the queries come from x, but keys and values could come from different sources, this is known as cross-attention. 

#scaled attention additionally divides wei by sqrt(head_size), this makes it so that when input QK are unit variance, wei will be unit variance too & softmax will stay diffuse 
#and not saturate too much

torch.Size([4, 8, 16])

In [47]:
k.var()

tensor(0.3164, grad_fn=<VarBackward0>)

In [48]:
q.var()

tensor(0.3386, grad_fn=<VarBackward0>)

In [49]:
wei.var()

tensor(0.0488, grad_fn=<VarBackward0>)

In [41]:
wei[0]

tensor([[0.0248, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0052, 0.0091, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0521, 0.0135, 0.2482, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3171, 0.0214, 0.1642, 0.1188, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0412, 0.0487, 0.1046, 0.0742, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1060, 0.5347, 0.2059, 0.1030, 0.7402, 0.0192, 0.0000, 0.0000],
        [0.4298, 0.3409, 0.1769, 0.2027, 0.0480, 0.8472, 0.2329, 0.0000],
        [0.0238, 0.0316, 0.1002, 0.5013, 0.0117, 0.1336, 0.7671, 1.0000]],
       grad_fn=<SelectBackward0>)

In [28]:
#we can achieve this bag of words effect with a simple matmul instead of the for loops from before: 
torch.manual_seed(42)
a = torch.ones(3,3)
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print("a=")
print(a)
print("--")
print("b=")
print(b)
print("--")
print("c=")
print(c)
print("--")

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])
--


In [29]:
#we can achieve this bag of words effect with a simple matmul instead of the for loops from before: 
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print("a=")
print(a)
print("--")
print("b=")
print(b)
print("--")
print("c=")
print(c)
print("--")
#The triangular mask has the effect of this matmul doing a vertical running sum.

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])
--


In [30]:
#normalizing the rows of a to sum to 1 has the effect of causing this matmul to not just do a vertical running sum, but a vertical running average
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a/torch.sum(a,1,keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print("a=")
print(a)
print("--")
print("b=")
print(b)
print("--")
print("c=")
print(c)
print("--")

#Scroll back up to see the vectorized version of our for loop bow calc! 

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
--
